<center><a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a></center>

<h1><center>ASSIGNMENT III - THE BATTLE OF NEIGHBOURHOODS</center></h1>

---
<h1><center>Part I - New York City & Toronto  comparison</center></h1>

### Import and download library

In [30]:
pip install BeautifulSoup4 

Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
print('Everything have been downloaded and imported')

Everything have been downloaded and imported


### Get a local copy in the repository of the Toronto data (Postcode, Borough, Neighbourhood)

In [34]:
#get a local copy of the Wikipedia article
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()
with open('List_of_postal_codes_of_Canada:_M.html', 'w') as fo:
    fo.write(article)

### Load data from repository

In [35]:
tables = pd.read_html('List_of_postal_codes_of_Canada:_M.html', header = 0, keep_default_na = False, )
tables = pd.DataFrame(tables[0])
tables.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Delete rows where Borough are not assigned / rearrange index

In [36]:
tables = tables[tables.Borough != 'Not assigned']
tables.reset_index(inplace = True)
tables = tables.drop(['index'], axis = 1)
tables.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Merge rows with same PostalCode and keep different Neighbourhood name separated with ','

In [37]:
tables = tables.groupby(["Postcode", "Borough"], as_index=False)["Neighbourhood"].agg(", ".join)
tables.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Replace not assigned Neighbourhood with Borough corresponding

In [38]:
tables.loc[tables.Neighbourhood == 'Not assigned', 'Neighbourhood'] = tables.Borough

In [39]:
tables[tables.Neighbourhood.str.contains("Not Assigned|Not assigned")]

,Postcode,Borough,Neighbourhood


### Upload data with longitude, latitude and postcode for Toronto and merge two tables 

In [40]:
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data
print('Data downloaded!')

--2019-08-29 00:22:11--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-08-29 00:22:11--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-08-29 00:22:12--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-08-29 00:22:12--  https://ibm.box.com/public/static/9afz

In [41]:
geo = pd.read_csv("Geospatial_data.csv")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
geo.columns = ['Postcode', 'Latitude', 'Longitude']

In [43]:
df_geo = pd.merge(tables, geo, on='Postcode', how='left')
df_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Copy dataframe into repository

In [44]:
df_geo.to_csv(r'Toronto_data.csv', index = False)

### Load data from repository for New York City

In [46]:
nyc = pd.read_csv('NHoodNameCentroids.csv')
nyc.head()

,the_geom,OBJECTID,Name,Stacked,AnnoLine1,AnnoLine2,AnnoLine3,AnnoAngle,Borough
0,POINT (-73.8472005205491 40.89470517661004),1,Wakefield,1,Wakefield,NaN,NaN,0,Bronx
1,POINT (-73.82993910812405 40.87429419303015),2,Co-op City,2,Co-op,City,NaN,0,Bronx
2,POINT (-73.82780644716419 40.88755567735082),3,Eastchester,1,Eastchester,NaN,NaN,0,Bronx
3,POINT (-73.90564259591689 40.895437426903875),4,Fieldston,1,Fieldston,NaN,NaN,0,Bronx
4,POINT (-73.91258546108577 40.89083449389134),5,Riverdale,1,Riverdale,NaN,NaN,0,Bronx


### Clean the dataframe

In [47]:
#Delete unwanted columns
nyc = nyc.drop(['OBJECTID', 'Stacked', 'AnnoLine1', 'AnnoLine2', 'AnnoLine3', 'AnnoAngle'], axis = 1)

#Rename columns
nyc.columns = ['Location', 'Neighborhood', 'Borough']

#Delete substring 'POINT' in columns Location
nyc['Location'] = nyc.Location.str.extract('.*\((.*)\)')

#Create columns Longitude
nyc['Longitude'] = nyc['Location'].str.split(' ').str[0]
nyc['Longitude'] = nyc['Longitude'].map(lambda x: str(x)[:10])

#Create columns Latitude
nyc['Latitude'] = nyc['Location'].str.split(' ').str[1]
nyc['Latitude'] = nyc['Latitude'].map(lambda x: str(x)[:9])

#Delete columns ocation and reindex dataframe
nyc = nyc.drop(['Location'], axis = 1)
nyc = nyc.reindex(columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'])
nyc.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847200
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887555,-73.827806
3,Bronx,Fieldston,40.895437,-73.905642
4,Bronx,Riverdale,40.890834,-73.912585


### Import and download library for mapping

In [48]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries downloaded and imported.')#

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

### Get the coordinates of Toronto and New-York City

In [53]:
address_TO = 'Toronto, TO'
address_NYC = 'New-York-City, NYC'

geolocator_TO = Nominatim(user_agent="toronto-map_data")
geolocator_NYC = Nominatim(user_agent="ny_explorer")

location_TO = geolocator_TO.geocode(address_TO)
latitude_TO = location_TO.latitude
longitude_TO = location_TO.longitude

location_NYC = geolocator_NYC.geocode(address_NYC)
latitude_NYC = location_NYC.latitude
longitude_NYC = location_NYC.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_TO, longitude_TO))
print('The geograpical coordinate of New-York-City are {}, {}.'.format(latitude_NYC, longitude_NYC))

The geograpical coordinate of Toronto are 43.6523873, -79.3835641.
The geograpical coordinate of New-York-City are 40.76365535, -73.7693715286312.


### Loading Foursquare API profile

In [54]:
CLIENT_ID = '3JHWEITDFXAALR1TXRJND12HEVZF2V11BENGPFLLDZHSG2YA' # your Foursquare ID
CLIENT_SECRET = 'OSMXBJNX3MX3NY3YMZCXPJ5BLPZMRIRUKMUO3DW4OY2SYXGO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3JHWEITDFXAALR1TXRJND12HEVZF2V11BENGPFLLDZHSG2YA
CLIENT_SECRET:OSMXBJNX3MX3NY3YMZCXPJ5BLPZMRIRUKMUO3DW4OY2SYXGO


### Limit of number of venues returned by Foursquare API and define radius

In [55]:
LIMIT = 100 # are API
radius = 500 # define radius

### Create the function getNearbyVenues to get names, categories and coordinates of nearby places

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the above function on each neighborhood and create a new dataframe called TO_venues and NYC_venues

In [57]:
#################
# IF error with 'groups', means I exceded the quoto so add change credential with my other accounts:

#CLIENT_ID = 'W5ARXECJQPAU0Q02KPJBSSF0AMJWTM0K1QU32XH5K4B04M04' # your Foursquare ID
#CLIENT_SECRET = 'NSAEZNNXEKDMW445PIX3JTRLQZBGBGIVW3VSVRCBO4TB4P0P' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
#################

TO_venues = getNearbyVenues(names=df_geo['Postcode'], latitudes=df_geo['Latitude'], longitudes=df_geo['Longitude'])
print("Function runned and new dataframes created")

Function runned and new dataframes created


In [69]:
#################
# IF error with 'groups', means I exceded the quoto so add change credential with my other accounts:

#CLIENT_ID = 'W5ARXECJQPAU0Q02KPJBSSF0AMJWTM0K1QU32XH5K4B04M04' # your Foursquare ID
#CLIENT_SECRET = 'NSAEZNNXEKDMW445PIX3JTRLQZBGBGIVW3VSVRCBO4TB4P0P' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
#################

NYC_venues = getNearbyVenues(names=nyc['Neighborhood'], latitudes=nyc['Latitude'], longitudes=nyc['Longitude'])
print("Function runned and new dataframes created")

Function runned and new dataframes created


In [58]:
TO_venues.head()

,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


Copy dataframe in repository

In [59]:
TO_venues.to_csv(r'Toronto_venues.csv', index = False)

In [29]:
NYC_venues = NYC_venues.rename(columns = {'PostalCode': 'Neighbourhood', 
                                          'PostalCode Latitude': 'Latitude', 
                                          'PostalCode Longitude': 'Longitude'})
NYC_venues.head()

,Neighbourhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847200,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847200,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847200,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847200,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847200,Dunkin',40.890459,-73.849089,Donut Shop


In [30]:
print('There are {} uniques categories for New-York-City.'.format(len(NYC_venues['Venue Category'].unique())))
print('There are {} uniques categories for Toronto.'.format(len(TO_venues['Venue Category'].unique())))

There are 428 uniques categories for New-York-City.
There are 274 uniques categories for Toronto.


### Let's create dataframe with each neighbourhood along with the top 10 most common venues for both cities

### New-York-City

In [ ]:
#################
def top_10(venues, num_top_venues):
    
    # one hot encoding
    onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

    # add neighborhood column back to dataframe
    onehot['Neighbourhood'] = venues['Neighbourhood'] 

    # move neighborhood column to the first column
    fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
    onehot = onehot[fixed_columns]

    grouped = onehot.groupby('Neighbourhood').mean().reset_index()

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighbourhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    Neighbourhood_venues_sorted = pd.DataFrame(columns=columns)
    Neighbourhood_venues_sorted['Neighbourhood'] = grouped['Neighbourhood']

    for ind in np.arange(grouped.shape[0]):
        Neighbourhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)
####################

In [31]:
# one hot encoding
NYC_onehot = pd.get_dummies(NYC_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NYC_onehot['Neighbourhood'] = NYC_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [NYC_onehot.columns[-1]] + list(NYC_onehot.columns[:-1])
NYC_onehot = NYC_onehot[fixed_columns]

NYC_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,...,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
NYC_onehot.shape

(10227, 429)

In [33]:
NYC_grouped = NYC_onehot.groupby('Neighbourhood').mean().reset_index()
NYC_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,...,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Annadale,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
1,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
2,Arlington,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
3,Arrochar,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
4,Arverne,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00,0.055556,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,Woodhaven,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.040000,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
289,Woodlawn,0.0,0.0,0.0,0.0,0.0,0.040000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
290,Woodrow,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0
291,Woodside,0.0,0.0,0.0,0.0,0.0,0.036585,0.0,0.0,0.012195,...,0.0,0.0,0.0,0.0,0.0,0.00,0.012195,0.0,0.0,0.0


In [34]:
NYC_grouped.shape

(293, 429)

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
NYC_venues_sorted = pd.DataFrame(columns=columns)
NYC_venues_sorted['Neighbourhood'] = NYC_grouped['Neighbourhood']

for ind in np.arange(NYC_grouped.shape[0]):
    NYC_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NYC_grouped.iloc[ind, :], num_top_venues)

NYC_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annadale,Cosmetics Shop,Dance Studio,Sports Bar,Restaurant,American Restaurant,Pharmacy,Train Station,Pizza Place,Diner,Bakery
1,Arden Heights,Pharmacy,Coffee Shop,Bus Stop,Pizza Place,Fish & Chips Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2,Arlington,Deli / Bodega,Grocery Store,American Restaurant,Bus Stop,Intersection,Boat or Ferry,Farm,Farmers Market,Fast Food Restaurant,Field
3,Arrochar,Bus Stop,Italian Restaurant,Deli / Bodega,Supermarket,Sandwich Place,Pizza Place,Hotel,Bagel Shop,Polish Restaurant,Outdoors & Recreation
4,Arverne,Surf Spot,Sandwich Place,Metro Station,Pizza Place,Thai Restaurant,Bed & Breakfast,Coffee Shop,Board Shop,Playground,Donut Shop


### Toronto

In [60]:
# one hot encoding
TO_onehot = pd.get_dummies(TO_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TO_onehot['PostalCode'] = TO_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [TO_onehot.columns[-1]] + list(TO_onehot.columns[:-1])
TO_onehot = TO_onehot[fixed_columns]

TO_onehot.head()

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
TO_onehot.shape

(2253, 275)

In [62]:
TO_grouped = TO_onehot.groupby('PostalCode').mean().reset_index()
TO_grouped

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,M9N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,M9P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,M9R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,M9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
TO_grouped.shape

(99, 275)

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
TO_venues_sorted = pd.DataFrame(columns=columns)
TO_venues_sorted['PostalCode'] = TO_grouped['PostalCode']

for ind in np.arange(TO_grouped.shape[0]):
    TO_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TO_grouped.iloc[ind, :], num_top_venues)

TO_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Rec Center
1,M1C,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant
2,M1E,Electronics Store,Mexican Restaurant,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Yoga Studio,Diner,Discount Store
3,M1G,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,M1H,Hakka Restaurant,Caribbean Restaurant,Bank,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Eastern European Restaurant,Dumpling Restaurant,Drugstore


Copy the dataframe into the repository

In [68]:
TO_venues_sorted.to_csv(r'Top_Toronto_Venues_Per_Neighbourhood.csv', index = False)

### Cluster neighbourhoods

### New-York-City

In [42]:
NYC_clustering = NYC_grouped.drop('Neighbourhood', axis = 1)

In [51]:
k_clusters = 5
NYC_kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(NYC_clustering)
NYC_kmeans.labels_

array([2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2,
       2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2,
       1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 4, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1,
       2, 2, 1, 1, 2, 2, 2], dtype=int32)

In [44]:
NYC_venues_sorted.insert(0, 'Cluster Labels', NYC_kmeans.labels_)

In [45]:
NYC_merged = nyc
NYC_merged.rename(columns = {'Neighborhood' : 'Neighbourhood'}, inplace = True)

In [46]:
NYC_merged = NYC_merged.merge(NYC_venues_sorted, left_on='Neighbourhood', right_on='Neighbourhood')
NYC_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847200,2,Pharmacy,Sandwich Place,Caribbean Restaurant,Ice Cream Shop,Dessert Shop,Gas Station,Food Truck,Donut Shop,Film Studio,Filipino Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,2,Bus Station,Baseball Field,Accessories Store,Ice Cream Shop,Gift Shop,Park,Grocery Store,Fast Food Restaurant,Chinese Restaurant,Mattress Store
2,Bronx,Eastchester,40.887555,-73.827806,2,Caribbean Restaurant,Bus Station,Metro Station,Diner,Cosmetics Shop,Pizza Place,Platform,Bowling Alley,Seafood Restaurant,Convenience Store
3,Bronx,Fieldston,40.895437,-73.905642,2,River,Playground,Plaza,Yoga Studio,Financial or Legal Service,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
4,Bronx,Riverdale,40.890834,-73.912585,2,Bus Station,Park,Bank,Plaza,Food Truck,Home Service,Playground,Factory,Falafel Restaurant,Farm


Visualize clusters on map

In [47]:
NYC_merged['Latitude'] = NYC_merged['Latitude'].astype('float64')
NYC_merged['Longitude'] = NYC_merged['Longitude'].astype('float64')

In [139]:
# create map of Toronto using latitude and longitude values
nyc_clustermap = folium.Map(location=[latitude_NYC, longitude_NYC], zoom_start=10)

# set color scheme for the clusters
rainbow = ['red','blue','green','yellow','black','pink', 'darkgreen','cyan','forestgreen']

# Plot the selected PostalCode to the map of Toronto and add marker
markers_colors = []
for lat, lon, post, cluster in zip(NYC_merged['Latitude'], NYC_merged['Longitude'], NYC_merged['Neighbourhood'], NYC_merged['Cluster Labels']):
    #print("post: {}; cluster: {}; lat: {}; lon: {}".format(post, cluster, lat, lon))
    label = folium.Popup(str(post) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-3],
        fill_opacity=0.7).add_to(nyc_clustermap)
       
nyc_clustermap

### Toronto

In [75]:
TO_clustering = TO_grouped.drop('PostalCode', axis = 1)

In [76]:
k_clusters = 5
TO_kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(TO_clustering)
TO_kmeans.labels_

array([0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       4, 0, 0, 0, 0, 4, 1, 0, 0, 0, 0], dtype=int32)

In [ ]:
TO_venues_sorted.insert(0, 'Cluster Labels', TO_kmeans.labels_)

In [71]:
TO_merged = df_geo

In [72]:
TO_merged = TO_merged.merge(TO_venues_sorted, left_on='Postcode', right_on='PostalCode')
TO_merged.drop(columns = ['PostalCode'], inplace = True)
TO_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Rec Center
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Electronics Store,Mexican Restaurant,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Yoga Studio,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Caribbean Restaurant,Bank,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Eastern European Restaurant,Dumpling Restaurant,Drugstore


In [1]:
# create map of Toronto using latitude and longitude values
to_clustermap = folium.Map(location=[latitude_TO, longitude_TO], zoom_start=10)

# set color scheme for the clusters
rainbow = ['red','blue','green','yellow','black','pink', 'darkgreen','cyan','forestgreen']

# Plot the selected PostalCode to the map of Toronto and add marker
markers_colors = []
for lat, lon, post, cluster in zip(TO_merged['Latitude'], TO_merged['Longitude'], TO_merged['Postcode'], TO_merged['Cluster Labels']):
    #print("post: {}; cluster: {}; lat: {}; lon: {}".format(post, cluster, lat, lon))
    label = folium.Popup(str(post) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-3],
        fill_opacity=0.7).add_to(to_clustermap)
       
to_clustermap

NameError: name 'folium' is not defined

### Examine Clusters

### New-York-City

Cluster 1

In [127]:
NYC_k1 = NYC_merged.loc[NYC_merged['Cluster Labels'] == 0].groupby('1st Most Common Venue').count()
NYC_k1 = NYC_k1.drop(NYC_k1.columns.to_series()[0:13], axis=1)
NYC_k1.columns = ['Recurrence']
NYC_k1 = NYC_k1.sort_values(by = ['Recurrence'], ascending = False)
NYC_k1.head()

,Recurrence
1st Most Common Venue,
Park,4


Cluster 2

In [128]:
NYC_k2 = NYC_merged.loc[NYC_merged['Cluster Labels'] == 1].groupby('1st Most Common Venue').count()
NYC_k2 = NYC_k2.drop(NYC_k2.columns.to_series()[0:13], axis=1)
NYC_k2.columns = ['Recurrence']
NYC_k2 = NYC_k2.sort_values(by = ['Recurrence'], ascending = False)
NYC_k2.head()

,Recurrence
1st Most Common Venue,
Deli / Bodega,19
Pizza Place,12
Bus Stop,5
Italian Restaurant,4
Sandwich Place,2


Cluster 3

In [129]:
NYC_k3 = NYC_merged.loc[NYC_merged['Cluster Labels'] == 2].groupby('1st Most Common Venue').count()
NYC_k3 = NYC_k3.drop(NYC_k3.columns.to_series()[0:13], axis=1)
NYC_k3.columns = ['Recurrence']
NYC_k3 = NYC_k3.sort_values(by = ['Recurrence'], ascending = False)
NYC_k3.head()

,Recurrence
1st Most Common Venue,
Italian Restaurant,22
Pizza Place,18
Coffee Shop,12
Bar,11
Chinese Restaurant,10


Cluster 4

In [130]:
NYC_k4 = NYC_merged.loc[NYC_merged['Cluster Labels'] == 3].groupby('1st Most Common Venue').count()
NYC_k4 = NYC_k4.drop(NYC_k4.columns.to_series()[0:13], axis=1)
NYC_k4.columns = ['Recurrence']
NYC_k4 = NYC_k4.sort_values(by = ['Recurrence'], ascending = False)
NYC_k4.head()

,Recurrence
1st Most Common Venue,
Locksmith,1


Cluster 5

In [131]:
NYC_k5 = NYC_merged.loc[NYC_merged['Cluster Labels'] == 4].groupby('1st Most Common Venue').count()
NYC_k5 = NYC_k5.drop(NYC_k5.columns.to_series()[0:13], axis=1)
NYC_k5.columns = ['Recurrence']
NYC_k5 = NYC_k5.sort_values(by = ['Recurrence'], ascending = False)
NYC_k5.head()

,Recurrence
1st Most Common Venue,
Beach,1


### Toronto

Cluster 1 

In [132]:
TO_k1 = TO_merged.loc[TO_merged['Cluster Labels'] == 0].groupby('1st Most Common Venue').count()
TO_k1 = TO_k1.drop(TO_k1.columns.to_series()[0:14], axis=1)
TO_k1.columns = ['Recurrence']
TO_k1 = TO_k1.sort_values(by = ['Recurrence'], ascending = False)
TO_k1.head()

,Recurrence
1st Most Common Venue,
Coffee Shop,21
Café,9
Park,8
Pizza Place,4
Clothing Store,3


Cluster 2

In [133]:
TO_k2 = TO_merged.loc[TO_merged['Cluster Labels'] == 1].groupby('1st Most Common Venue').count()
TO_k2 = TO_k2.drop(TO_k2.columns.to_series()[0:14], axis=1)
TO_k2.columns = ['Recurrence']
TO_k2 = TO_k2.sort_values(by = ['Recurrence'], ascending = False)
TO_k2.head()

,Recurrence
1st Most Common Venue,
Park,4
Airport,1
Bus Stop,1
River,1


Cluster 3

In [134]:
TO_k3 = TO_merged.loc[TO_merged['Cluster Labels'] == 2].groupby('1st Most Common Venue').count()
TO_k3 = TO_k3.drop(TO_k3.columns.to_series()[0:14], axis=1)
TO_k3.columns = ['Recurrence']
TO_k3 = TO_k3.sort_values(by = ['Recurrence'], ascending = False)
TO_k3.head()

,Recurrence
1st Most Common Venue,
Bar,1


Cluster 4

In [135]:
TO_k4 = TO_merged.loc[TO_merged['Cluster Labels'] == 3].groupby('1st Most Common Venue').count()
TO_k4 = TO_k4.drop(TO_k4.columns.to_series()[0:14], axis=1)
TO_k4.columns = ['Recurrence']
TO_k4 = TO_k4.sort_values(by = ['Recurrence'], ascending = False)
TO_k4.head()

,Recurrence
1st Most Common Venue,
Park,1
Playground,1
Spa,1


Cluster 5

In [136]:
TO_k5 = TO_merged.loc[TO_merged['Cluster Labels'] == 4].groupby('1st Most Common Venue').count()
TO_k5 = TO_k5.drop(TO_k5.columns.to_series()[0:14], axis=1)
TO_k5.columns = ['Recurrence']
TO_k5 = TO_k5.sort_values(by = ['Recurrence'], ascending = False)
TO_k5.head()

,Recurrence
1st Most Common Venue,
Baseball Field,3
Garden,1
